## BRONZE TO SILVER LAYER

### Silver Layer - Stock Price History


In [1]:
# Importing Common Utility Function
import re

import pandas as pd

from StockETL import GlobalPath

In [2]:
# Import necessary libraries and utility functions
%run ../COMMON/common_utility.ipynb

USERNAME = 'ptprashanttripathi'


In [3]:
# Instantiate GlobalPath
stockdata_bronze_layer_path = GlobalPath("DATA/BRONZE/StockData")
stockprice_silver_file_path = GlobalPath("DATA/SILVER/StockPrice/StockPrice_data.csv")
stockprice_silver_schema_file_path = GlobalPath(
    "CONFIG/DATA_CONTRACTS/SILVER/StockPrice.json"
)

### Data Processing

- Initialize an empty list to store DataFrames.
- Read and concatenate data from multiple CSV files.


In [4]:
# Initialize an empty list to store individual stock price DataFrames
df_stock_price_list = []

# Generate file paths for available CSV files in the Bronze layer
file_paths = check_files_availability(stockdata_bronze_layer_path, file_pattern="*.csv")

# Loop through the list of CSV files in the folder
for file_path in file_paths:
    print(f"Processing file => {file_path}")
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    # Extract the stock symbol by removing the date pattern from the file name
    df["symbol"] = re.sub(r"_\d{4}_\d{2}\.csv", "", file_path.name)
    # Append the DataFrame to the list of DataFrames
    df_stock_price_list.append(df)

# Concatenate all individual DataFrames into one combined DataFrame
df = pd.concat(df_stock_price_list, ignore_index=True)
# Standardize column names by replacing punctuation
df = replace_punctuation_from_columns(df)
# Remove columns that contain only NA values
df.dropna(how="all", axis=1, inplace=True)

Number of Files Detected => 1676
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/KPITTECH_2024_08.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/CTL_2024_12.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/TATACHEM_2022_07.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/ADANIPORTS_2023_03.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/HAPPSTMNDS_2024_03.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/KPEL_2024_02.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/CTL_2022_07.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/HAPPSTMNDS_2024_08.csv
Processing file => /home/runner/work/PortfolioTracker/Portfolio

Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/RTNPOWER_2022_09.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/SBI_LONG_TERM_EQUITY_FUND_DIRECT_PLAN_GROWTH_2025_01.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/HAPPSTMNDS_2022_12.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/BECTORFOOD_2022_04.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/VISESHINFO_2023_03.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/TATAMOTORS_2021_10.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/GITARENEW_2023_05.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/BCG_2023_09.csv
Processing file => /home/runner/work/PortfolioTr

Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/VOLTAS_2021_09.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/HDFC_INDEX_FUNDNIFTY_50_PLAN_DIRECT_PLAN_2022_07.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/CRESSAN_2022_09.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/FCSSOFT_2023_01.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/VISESHINFO_2024_01.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/CTL_2023_07.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/VOLTAS_2022_02.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/RTNPOWER_2023_01.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTr

Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/TATAPOWER_2023_03.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/KPEL_2024_06.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/RTNINDIA_2023_07.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/SYNCOMF_2024_05.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/ZOMATO_2022_06.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/GITARENEW_2022_03.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/SYNCOMF_2024_09.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/HDFC_INDEX_FUNDNIFTY_50_PLAN_DIRECT_PLAN_2024_11.csv
Processing file => /home/runner/work/PortfolioTracker/Portfoli

Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/RTNPOWER_2023_02.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/RTNPOWER_2022_12.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/CTL_2023_12.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/ICICIB22_2022_05.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/RTNINDIA_2022_05.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/URJA_2023_02.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/YESBANK_2021_09.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/TTML_2022_09.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/KPITTECH_

...

Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/TATAPOWER_2022_08.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/AWL_2024_03.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/KPITTECH_2024_09.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/VISESHINFO_2024_06.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/BECTORFOOD_2022_03.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/ZOMATO_2024_10.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/CTL_2021_12.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/GRAVITA_2023_12.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/LLO

Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/VISESHINFO_2023_10.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/ICICIB22_2022_11.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/BECTORFOOD_2023_08.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/KPITTECH_2024_03.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/TATAMOTORS_2021_07.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/HFCL_2024_05.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/BCG_2022_08.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/RTNINDIA_2023_02.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockDat

Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/BANKBARODA_2021_10.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/SUZLON_2024_02.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/ICICIB22_2022_02.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/VIKASLIFE_2022_01.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/VIKASLIFE_2023_07.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/AWL_2022_12.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/TCS_2024_04.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/TTML_2024_06.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/TATAMOT

Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/CTL_2022_05.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/BECTORFOOD_2020_12.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/PNB_2020_12.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/URJA_2021_12.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/VIKASLIFE_2023_01.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/YESBANK_2021_04.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/WIPRO_2023_06.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/YESBANK_2021_01.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/DSP_ELSS_TA

Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/TATACHEM_2021_09.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/URJA_2022_01.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/JPPOWER_2024_09.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/MIRAE_ASSET_ELSS_TAX_SAVER_FUND_DIRECT_PLAN_GROWTH_2024_08.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/SWORDEDGE_2022_05.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/IRCTC_2024_12.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/RAGHUSYN_2023_09.csv
Processing file => /home/runner/work/PortfolioTracker/PortfolioTracker/DATA/BRONZE/StockData/RTNPOWER_2024_03.csv
Processing file => /home/runner/work/PortfolioTracker

### Final Processing and Export

- Sort the DataFrame by stock name and date.
- Save the processed data as a CSV file in the Silver layer.

In [5]:
# Convert datetime to date string
df["date"] = pd.to_datetime(df["date"]).dt.date

# Align Datafame with DataContract
df = align_with_datacontract(df, stockprice_silver_schema_file_path)

# Save the result as a CSV file
df.to_csv(stockprice_silver_file_path, index=None)

print("SILVER Layer CSV file for Stock Price history successfully created at =>")
print(stockprice_silver_file_path)
# Log the DataFrame debug
df.info()

DataContract loaded from => /home/runner/work/PortfolioTracker/PortfolioTracker/CONFIG/DATA_CONTRACTS/SILVER/StockPrice.json
SILVER Layer CSV file for Stock Price history successfully created at =>
/home/runner/work/PortfolioTracker/PortfolioTracker/DATA/SILVER/StockPrice/StockPrice_data.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32264 entries, 0 to 32263
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    32264 non-null  string 
 1   symbol  32264 non-null  string 
 2   open    32264 non-null  float64
 3   high    32264 non-null  float64
 4   low     32264 non-null  float64
 5   close   32264 non-null  float64
 6   volume  29757 non-null  float64
dtypes: float64(5), string(2)
memory usage: 1.7 MB
